In [32]:
from collections import defaultdict
from ipywidgets import IntProgress
from IPython.display import display

f = open("input.txt", "r")
#f = open("input.txt", "r")
l = f.read().split("\n")[:-1]
#print(l)


class MyValve:
    def __init__(self,name,value,tars):
        self.name = name
        self.value = int(value)
        self.tars = tars.copy()
        self.pathDict = defaultdict(str)
        self.activateAt = -1
        self.pathTo = []

class MyNode:
    def __init__(self,valve):
        self.valve = valve
        self.g= 0
        
        self.pred = None
    def getCost(self):
        return self.g
    def compare(self,other):
        return self.valve.name == other.valve.name

def elementInList(elem,nodeList):
    for i in range(0,len(nodeList)):
        if nodeList[i].compare(elem):
            return True
    return False
           
def checkBetter(elem,nodeList):
    for e in nodeList:
        if elem.compare(e) and elem.getCost() > e.getCost():
            return True
    return False

def insertOrUpdate(elem,nodeList):
    for i in range(0,len(nodeList)):
        if nodeList[i].compare(elem):
            if nodeList[i].getCost() < elem.getCost():
                nodeList[i] = elem
            return
    nodeList.append(elem)
def expand(node,openList,closedList):
    sucList= []
    #print("mmmm",node)
    for x in node.valve.pathTo:
        tmp  = MyNode(x)
        sucList.append(tmp)
    #print("ssss",sucList)
    for e in sucList:
        #print("eeee",e)
        if elementInList(e,closedList):continue
        if checkBetter(e,openList): continue

        e.pred = node

        insertOrUpdate(e,openList)
        
def fromNodes(a,n):
    return [x for x in n if x.name == a][0]
def astar(start, dest):
    
    closedList =[]
    startNode = MyNode(start)
    endNode = MyNode(dest)
    openList = [startNode]

    while(len(openList)>0):
        m = openList[0]
        openList = openList[1:]
        if m.compare(endNode):
            return m
        closedList.append(m)
        
        expand(m,openList,closedList)
    return None

        
nodeList = []        
nodeStrings = []
for x in l:
    #print(x)
    _name = x.split(" ")[1]
    _value = x.split(" ")[4].split("=")[1][:-1]
    _tars = x.replace(",","").split(" ")[9:]
    #print(_name,_value,_tars)
    tmp = MyValve(_name,_value,_tars)
    nodeList.append(tmp)
    nodeStrings.append(_name)
for x in nodeList:
    #print("=====",x.name,"======")
    for k in x.tars:
        tmp = [j for j in nodeList if j.name == k][0]
        x.pathTo.append(tmp)
graph = defaultdict(list)
graphValues = defaultdict(int)
nns = defaultdict(object)
for i in range(0,len(nodeList)):
    #print(i)

    cur = nodeList[i]
    graphValues[nodeList[i].name] = nodeList[i].value
    nns[nodeList[i].name] = nodeList[i]
    for x in range(0,len(nodeList)):
        if x == i: continue
        nn = astar(cur,nodeList[x])
        #print(nn)
        nnList = []
        while(nn.pred != None):
            nnList.append(nn.valve.name)
                
            nn = nn.pred
        nnList = list(reversed(nnList))
        graph[nodeList[x].name] = nodeList[x].tars
        cur.pathDict[nodeList[x].name] =nnList
        #print("from", cur.name, "to", nodeList[x].name,cur.pathDict[nodeList[x].name])
   




In [33]:

def evalRoute(r):

    v = 0
    t = 26
    node = nns[r[0]]
    for i in range(1,len(r)):
        t -= len(node.pathDict[r[i]]) +1
        node = nns[r[i]]
        v += t*node.value
        #print(r[i],v,t)
    return v
        
        


In [34]:

pts = []
def dfs(visited, node,time):  
    if time <= 0:
        return 
    for key in node.pathDict:
        dist = len(node.pathDict[key])
        if dist +1 >= time:
            continue
        if graphValues[key] == 0:
            continue
        if key in visited:
            continue
        _node = nns[key]    

        dfs(visited+[key], _node, (time -dist -1))
    pts.append([visited])


dfs([],  nns['AA'],26)
evalrt = defaultdict(int)
st = defaultdict(set)


mx = 0
mxR = []
for i in range(0,len(pts)):
    if evalrt[i] == 0:
         evalrt[i] =  evalRoute(['AA']+pts[i][0])
         st[i] = set(pts[i][0])

print(len(pts)*len(pts))
pBar  = IntProgress(min=0, max=len(pts)*len(pts)) # instantiate the bar
display(pBar)
issdct = defaultdict(set)
issseen = defaultdict(int)
for i in range(0,len(pts)):

    for n in range(0,len(pts)):
        if n == i:
            continue
        if (i+1)*n %1000 == 0:
            
            pBar.value+=1000

        if evalrt[i] == 0:
            evalrt[i] =  evalRoute(['AA']+pts[i][0])
            a = evalrt[i]
            st[i] = set(pts[i][0])
            sx = st[i]
        else:
            
            a = evalrt[i]
            sx = st[i]
        if evalrt[n] == 0:
            evalrt[n] =  evalRoute(['AA']+pts[n][0])
            b = evalrt[n]
            st[n] = set(pts[n][0])
            sy = st[n]
        else:
            b = evalrt[n]
            sy = st[n]
        if sy == set() or sx ==set():
            continue

        if a+b<=mx:
            continue
        
        if issdct[i,n] == set() and issseen[i,n] ==0:
            iss = sx.intersection(sy)
            issdct[i,n] = iss
            issdct[n,i] = iss
            issseen[i,n] = 1
            issseen[n,i] = i
        else:
            iss = issdct[i,n]
            

        if iss == set() and sy != set() and sx != set():
            if a+b > mx:
                mx = a+b
                mxR = [pts[i][0],pts[n][0]]

print(mx)
print(mxR)


1802511936


IntProgress(value=0, max=1802511936)

2216
[['UP', 'AR', 'EJ', 'OW', 'VO'], ['XN', 'GL', 'VR', 'AH', 'HZ', 'JF']]
